# Google's Gemini 2.0 

## Information Extraction: Tabular Data, continued

In [1]:
import os

GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]

In [2]:
import json
import numpy as np
import pandas as pd

from google import genai
from google.genai import types

from pydantic import BaseModel, Field

In [3]:
# Create a client
client = genai.Client(api_key=GEMINI_API_KEY)

# Specify the model you are going to use
model_id =  "gemini-2.0-flash" # or "gemini-2.0-flash-lite-preview-02-05"  , "gemini-2.0-pro-exp-02-05"

----

In [4]:
with open("samples/saintmarc-hd_20250313.pdf", "rb") as f:
      file_bytes = f.read()

Feb2025_pdf = types.Part.from_bytes(
    data=file_bytes,
    mime_type='application/pdf',
)

In [5]:
SYSTEM_LEVEL_INSTRUCTION = (
    "You are perfectly bilingual, fluent in both Japanese and English. "    
    "You are also an expert data analyst, specializing in information extraction from PDFs.  "
    "You especially enjoy parsing out tabular data, always being completely accurate when extracting table parts "
    "such as the row and column headers, and table cells. "
    "You always understand the layout of a table, and know how to return empty values."
).strip()

In [6]:
initial_prompt="""
Retrieve the entire contents for the 月次売上情報 table in the given PDF.

""".strip()

In [7]:
%%time

response = client.models.generate_content(
        model=model_id,

        config=types.GenerateContentConfig(
            system_instruction=SYSTEM_LEVEL_INSTRUCTION,
            response_mime_type='application/json', 
        ),
        
        contents=[
            Feb2025_pdf,            
            initial_prompt,
        ]
    )

print(response.text)

[
  {
    "年度": "2022",
    "4月": "118.9",
    "5月": "144.0",
    "6月": "126.3",
    "7月": "110.7",
    "8月": "124.0",
    "9月": "127.2",
    "上半期": "124.5",
    "10月": "115.7",
    "11月": "107.4",
    "12月": "106.1",
    "1月": "122.5",
    "2月": "140.1",
    "3月": "120.5",
    "通期": "120.7",
    "指標": "昨年対比",
    "種類": "全店売上 (%)"
  },
  {
    "年度": "2023",
    "4月": "116.0",
    "5月": "110.7",
    "6月": "109.5",
    "7月": "117.6",
    "8月": "119.1",
    "9月": "114.3",
    "上半期": "114.6",
    "10月": "106.5",
    "11月": "108.6",
    "12月": "108.8",
    "1月": "108.1",
    "2月": "107.9",
    "3月": "110.3",
    "通期": "111.3",
    "指標": "昨年対比",
    "種類": "全店売上 (%)"
  },
  {
    "年度": "2024",
    "4月": "102.6",
    "5月": "102.4",
    "6月": "109.9",
    "7月": "100.7",
    "8月": "106.6",
    "9月": "105.6",
    "上半期": "104.6",
    "10月": "98.8",
    "11月": "104.5",
    "12月": "101.8",
    "1月": "101.2",
    "2月": "102.5",
    "3月": null,
    "通期": null,
    "指標": "昨年対比",
    "種類": "全店売上 (%)"
  

In [8]:
table_prompt="""
Retrieve the entire contents for the 月次売上情報 table in the given PDF.

This table has the following 15 columns:
- 年度
- 4月
- 5月
- 6月
- 7月
- 8月
- 9月
- 上半期
- 10月
- 11月
- 12月
- 1月
- 2月
- 3月
- 通期

There are 2 major row groupings:
- 全店売上
- 既存店売上

Ignore the unneeded row header for 昨年対比（％）.

For each major row group listed above, there are 3 rows for:
- yyyy for 2 years prior
- yyyy for the previous year
- yyyy for the current year

If any of the 15 column values in a row is empty, please represent this with a null value.

""".strip()

----

In [11]:
def extract_structured_data(model_id:str, prompt:str, file_part:types.Part, response_schema:BaseModel):
    """Given the Google Gen AI model,
       a prompt for the model,
       the raw bytes from a PDF file (max. 20MB!) which we send inline with our request,
       and a schema for the structured response we wish to obtain:

       Return a structured response (MIME type application/json) for the given prompt.
    """
    response = client.models.generate_content(
        model=model_id,

        config=types.GenerateContentConfig(
            system_instruction=SYSTEM_LEVEL_INSTRUCTION,
            response_mime_type='application/json', 
            response_schema=response_schema,
            seed=42
        ),
        
        contents=[
            file_part,            
            prompt,
        ]
    )
    
    # Convert the response to the Pydantic model and return it
    #print(response)
    return response.parsed

In [28]:
class TableRow(BaseModel):
    uriage: str = Field(description="売上 type, one of: 全店売上 or 既存店売上")

    年度: str = Field(description=("the values for in this row's 年度 column. If this value is empty, represent with a null value."))
    Apr: str = Field(description=("the values for in this row's 4月 column. If this value is empty, represent with a null value."))
    May: str = Field(description=("the values for in this row's 5月 column. If this value is empty, represent with a null value."))
    Jun: str = Field(description=("the values for in this row's 6月 column. If this value is empty, represent with a null value."))
    Jul: str = Field(description=("the values for in this row's 7月 column. If this value is empty, represent with a null value."))
    Aug: str = Field(description=("the values for in this row's 8月 column. If this value is empty, represent with a null value."))
    Sep: str = Field(description=("the values for in this row's 9月 column. If this value is empty, represent with a null value."))
    上半期: str = Field(description=("the values for in this row's 上半期 column. If this value is empty, represent with a null value."))
    Oct: str = Field(description=("the values for in this row's 10月 column. If this value is empty, represent with a null value."))
    Nov: str = Field(description=("the values for in this row's 11月 column. If this value is empty, represent with a null value."))
    Dec: str = Field(description=("the values for in this row's 12月 column. If this value is empty, represent with a null value."))
    Jan: str = Field(description=("the values for in this row's 1月 column. If this value is empty, represent with a null value."))
    Feb: str = Field(description=("the values for in this row's 2月 column. If this value is empty, represent with a null value."))
    Mar: str = Field(description=("the values for in this row's 3月 column. If this value is empty, represent with a null value."))
    通期: str = Field(description=("the values for in this row's 通期 column. If this value is empty, represent with a null value."))

class Table(BaseModel):
    rows: list[TableRow] = Field(description="List of TableRow objects that make up the data in the table")

In [30]:
%%time

table_data = extract_structured_data(model_id, table_prompt, Feb2025_pdf, Table)
#table_data.model_dump()['rows']

CPU times: user 12.2 ms, sys: 0 ns, total: 12.2 ms
Wall time: 7.64 s


[{'uriage': '全店売上',
  '年度': '2022',
  'Apr': '118.9',
  'May': '144.0',
  'Jun': '126.3',
  'Jul': '110.7',
  'Aug': '124.0',
  'Sep': '127.2',
  '上半期': '124.5',
  'Oct': '115.7',
  'Nov': '107.4',
  'Dec': '106.1',
  'Jan': '122.5',
  'Feb': '140.1',
  'Mar': '120.5',
  '通期': '120.7'},
 {'uriage': '全店売上',
  '年度': '2023',
  'Apr': '116.0',
  'May': '110.7',
  'Jun': '109.5',
  'Jul': '117.6',
  'Aug': '119.1',
  'Sep': '114.3',
  '上半期': '114.6',
  'Oct': '106.5',
  'Nov': '108.6',
  'Dec': '108.8',
  'Jan': '108.1',
  'Feb': '107.9',
  'Mar': '110.3',
  '通期': '111.3'},
 {'uriage': '全店売上',
  '年度': '2024',
  'Apr': '102.6',
  'May': '102.4',
  'Jun': '109.9',
  'Jul': '100.7',
  'Aug': '106.6',
  'Sep': '105.6',
  '上半期': '104.6',
  'Oct': '98.8',
  'Nov': '104.5',
  'Dec': '101.8',
  'Jan': '101.2',
  'Feb': '102.5',
  'Mar': 'null',
  '通期': 'null'},
 {'uriage': '既存店売上',
  '年度': '2022',
  'Apr': '115.1',
  'May': '126.0',
  'Jun': '122.8',
  'Jul': '111.5',
  'Aug': '124.8',
  'Sep': '12

In [32]:
tmp_df = pd.DataFrame(table_data.model_dump()['rows'])
tmp_df

,uriage,年度,Apr,May,Jun,Jul,Aug,Sep,上半期,Oct,Nov,Dec,Jan,Feb,Mar,通期
0,全店売上,2022,118.9,144.0,126.3,110.7,124.0,127.2,124.5,115.7,107.4,106.1,122.5,140.1,120.5,120.7
1,全店売上,2023,116.0,110.7,109.5,117.6,119.1,114.3,114.6,106.5,108.6,108.8,108.1,107.9,110.3,111.3
2,全店売上,2024,102.6,102.4,109.9,100.7,106.6,105.6,104.6,98.8,104.5,101.8,101.2,102.5,null,null
3,既存店売上,2022,115.1,126.0,122.8,111.5,124.8,127.4,120.9,115.8,107.0,106.3,123.6,143.7,123.8,119.6
4,既存店売上,2023,119.7,114.6,113.8,120.7,122.9,117.1,118.2,110.3,113.1,113.3,113.2,112.8,115.1,115.5
5,既存店売上,2024,107.1,106.3,113.9,105.2,110.5,110.3,108.8,103.8,109.2,104.9,104.0,104.6,null,null


In [52]:
tmp_df.iloc[:,2:].values

array([['118.9', '144.0', '126.3', '110.7', '124.0', '127.2', '124.5',
        '115.7', '107.4', '106.1', '122.5', '140.1', '120.5', '120.7'],
       ['116.0', '110.7', '109.5', '117.6', '119.1', '114.3', '114.6',
        '106.5', '108.6', '108.8', '108.1', '107.9', '110.3', '111.3'],
       ['102.6', '102.4', '109.9', '100.7', '106.6', '105.6', '104.6',
        '98.8', '104.5', '101.8', '101.2', '102.5', 'null', 'null'],
       ['115.1', '126.0', '122.8', '111.5', '124.8', '127.4', '120.9',
        '115.8', '107.0', '106.3', '123.6', '143.7', '123.8', '119.6'],
       ['119.7', '114.6', '113.8', '120.7', '122.9', '117.1', '118.2',
        '110.3', '113.1', '113.3', '113.2', '112.8', '115.1', '115.5'],
       ['107.1', '106.3', '113.9', '105.2', '110.5', '110.3', '108.8',
        '103.8', '109.2', '104.9', '104.0', '104.6', 'null', 'null']],
      dtype=object)

In [55]:
row_multi = pd.MultiIndex.from_frame(tmp_df.iloc[:,:2], names=None)

columns = tmp_df.columns[2:]

df = pd.DataFrame(
    tmp_df.iloc[:,2:].values,
    index=row_multi,
    columns=columns
)
df.replace('null', '', inplace=True)
df

Apr    May    Jun    Jul    Aug    Sep    上半期    Oct    Nov  \
uriage 年度                                                                    
全店売上   2022  118.9  144.0  126.3  110.7  124.0  127.2  124.5  115.7  107.4   
       2023  116.0  110.7  109.5  117.6  119.1  114.3  114.6  106.5  108.6   
       2024  102.6  102.4  109.9  100.7  106.6  105.6  104.6   98.8  104.5   
既存店売上  2022  115.1  126.0  122.8  111.5  124.8  127.4  120.9  115.8  107.0   
       2023  119.7  114.6  113.8  120.7  122.9  117.1  118.2  110.3  113.1   
       2024  107.1  106.3  113.9  105.2  110.5  110.3  108.8  103.8  109.2   

               Dec    Jan    Feb    Mar     通期  
uriage 年度                                       
全店売上   2022  106.1  122.5  140.1  120.5  120.7  
       2023  108.8  108.1  107.9  110.3  111.3  
       2024  101.8  101.2  102.5                
既存店売上  2022  106.3  123.6  143.7  123.8  119.6  
       2023  113.3  113.2  112.8  115.1  115.5  
       2024  104.9  104.0  104.6

Index(['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', '上半期', 'Oct', 'Nov', 'Dec',
       'Jan', 'Feb', 'Mar', '通期'],
      dtype='object')

In [ ]:
row_multi = pd.MultiIndex.from_tuples(
    zip(tmp_df.iloc[:,:2].values, tmp_df.iloc[:,0].values)
)

columns = tmp_df.columns[1:-1]

df = pd.DataFrame(
    data,
    index=row_multi,
    columns=columns
)
df.replace(np.nan, '', inplace=True)
df

----

In [9]:
%%time

response = client.models.generate_content(
        model=model_id,

        config=types.GenerateContentConfig(
            system_instruction=SYSTEM_LEVEL_INSTRUCTION,
            response_mime_type='application/json', 
        ),
        
        contents=[
            Feb2025_pdf,            
            table_prompt,
        ]
    )

#print(response.text)

CPU times: user 5.96 ms, sys: 218 μs, total: 6.18 ms
Wall time: 7.25 s


In [10]:
data = json.loads(response.text)

tmp_df = pd.DataFrame(data)

tmp_df

,年度,4月,5月,6月,7月,8月,9月,上半期,10月,11月,12月,1月,2月,3月,通期,row_header
0,2022,118.9,144.0,126.3,110.7,124.0,127.2,124.5,115.7,107.4,106.1,122.5,140.1,120.5,120.7,全店売上
1,2023,116.0,110.7,109.5,117.6,119.1,114.3,114.6,106.5,108.6,108.8,108.1,107.9,110.3,111.3,全店売上
2,2024,102.6,102.4,109.9,100.7,106.6,105.6,104.6,98.8,104.5,101.8,101.2,102.5,None,None,全店売上
3,2022,115.1,126.0,122.8,111.5,124.8,127.4,120.9,115.8,107.0,106.3,123.6,143.7,123.8,119.6,既存店売上
4,2023,119.7,114.6,113.8,120.7,122.9,117.1,118.2,110.3,113.1,113.3,113.2,112.8,115.1,115.5,既存店売上
5,2024,107.1,106.3,113.9,105.2,110.5,110.3,108.8,103.8,109.2,104.9,104.0,104.6,None,None,既存店売上


In [ ]:
data = json.loads(response.text)

tmp_df = pd.DataFrame(data)

data = tmp_df.iloc[:,1:-1].copy().values

row_multi = pd.MultiIndex.from_tuples(
    zip(tmp_df.iloc[:,-1].values, tmp_df.iloc[:,0].values)
)

columns = tmp_df.columns[1:-1]

df = pd.DataFrame(
    data,
    index=row_multi,
    columns=columns
)
df.replace(np.nan, '', inplace=True)
df

<hr width=40%/>

In [ ]:
with open("samples/saintmarc-hd_20250213.pdf", "rb") as f:
      file_bytes = f.read()

Jan2025_pdf = types.Part.from_bytes(
    data=file_bytes,
    mime_type='application/pdf',
)

In [ ]:
%%time

response = client.models.generate_content(
        model=model_id,

        config=types.GenerateContentConfig(
            system_instruction=SYSTEM_LEVEL_INSTRUCTION,
            response_mime_type='application/json', 
        ),
        
        contents=[
            Jan2025_pdf,            
            table_prompt,
        ]
    )

print(response.text)

In [ ]:
data = json.loads(response.text)
tmp_df = pd.DataFrame(data)

tmp_df

In [ ]:
data = json.loads(response.text)

tmp_df = pd.DataFrame(data)

data = tmp_df.iloc[:,1:-1].copy().values

row_multi = pd.MultiIndex.from_tuples(
    zip(tmp_df.iloc[:,-1].values, tmp_df.iloc[:,0].values)
)

columns = tmp_df.columns[1:-1]

df = pd.DataFrame(
    data,
    index=row_multi,
    columns=columns
)
df.replace(np.nan, '', inplace=True)
df

----

In [ ]:
# housekeeping...
for f in client.files.list():
    client.files.delete(name=f.name)